# Dyanmic-Dialo

Making a dynamic slot filling dialogue system

## creating and identifying slot functions

In [1]:
# as extension task add the slot type
def create_slot(slot_name, slot_model):
    slot_model[slot_name] = "unfilled"

def fill_slot(slot_name, slot_model, slot_value):
    slot_model[slot_name] = slot_value

In [ ]:
create_slot_func = {
    "name": "create_slot",
    "description": "function creates a slots based on a user query",
    "parameters": {
        "type": "object",
        "properties": {
            "slots_to_add": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "name of the slot"
                },
                "description": "list of slots to add to accomplish what the user wants"
            }
        },
        "required": ["slot_name"]
    }
}

fill_slot_func = {
    "name": "fill_slot",
    "description": "function fills a slot name with slot value",
    "parameters": {
        "type": "object",
        "properties": {
            "slots_to_fill":{
                "type": "array",
                "items":{
                    "type": "object",
                        "properties": {
                        "slot_name":{
                            "type": "string",
                            "description": "name of the slot"
                        },
                        "slot_value":{
                            "type": "string",
                            "description": "value of the slot"
                        }
                    }
                },
                "description": "list of slots to fill with their values"
            }
        },
        "required": ["slot_name", "slot_value"]
    }
}

In [3]:
create_slot_prompt = '''
    You are an assistant that can determine whether to create a slot.
    You are given a conversation context and a slot model.
    The slot model is a list of slot names that already exist. 
    A slot name corresponds to a piece of information that the user can provide.
    If the user asks a question or requests for information or an action that is not in the slot model then call create_slot_func and pass in a list of slot names designating the information the system would need answered by the user to accomplish what they request.
    If necessary you can add multiple slots representing the different pieces of information the user would need to provide.
    Use the entire conversation context as context but only take an action based on the last user message.
    Do not create a slot if the user is asking for information that is already in the slot model.
    If it is not necessary to create a slot then do nothing and do not call create_slot_func.
'''

fill_slot_prompt = '''
    You are an assistant that can determine whether to fill a slot.
    You are given a conversation context and a slot model.
    The slot model is a list of slot names that need to be filled. 
    A slot name corresponds to a piece of information that the user can provide.
    If the user answers with information answers a topic in the slot model then call fill_slot_func with the slot name and value.
    Use the entire conversation context as context but only take an action based on the last user message.
    Do not fill a slot if the user is not answering a question or providing information that is in the slot model.
    If it is not necessary to fill a slot then do nothing and do not call fill_slot_func.
'''

In [ ]:
from openai import OpenAI
import json

def check_create_slot(convo_context, slot_model, remaining_slots):
    tools_in = [
        {
            "type": "function",
            "function": create_slot_func,
        }
    ]
    
    user_prompt = f'''
        Here is the conversation context:
        {convo_context}
        Here are the slots that already exist:
        {", ".join(remaining_slots)}
    '''
    
    msgs_in = [
        {"role": "system", "content": create_slot_prompt},
        {"role": "user", "content": user_prompt}
    ]
        
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=msgs_in,
        tools=tools_in,
        temperature=0.3
    )
    
    tool_choice = completion.choices[0].message.tool_calls[0] if completion.choices[0].message.tool_calls else None
    if tool_choice:
        arguments = json.loads(tool_choice.function.arguments)
        for slot in arguments["slots_to_add"]:
            create_slot(slot, slot_model)
        return True
    return False

In [ ]:
def check_fill_slot(convo_context, slot_model, remaining_slots):
    tools_in = [
        {
            "type": "function",
            "function": fill_slot_func,
        }
    ]
    
    user_prompt = f'''
        Here is the conversation context:
        {convo_context}
        Here are the slots that need to be filled:
        {", ".join(remaining_slots)}
    '''
    
    msgs_in = [
        {"role": "system", "content": fill_slot_prompt},
        {"role": "user", "content": user_prompt}
    ]
        
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=msgs_in,
        tools=tools_in,
        temperature=0.3
    )
    
    tool_choice = completion.choices[0].message.tool_calls[0] if completion.choices[0].message.tool_calls else None

    if tool_choice:
        arguments = json.loads(tool_choice.function.arguments)
        for elem in arguments["slots_to_fill"]:
            fill_slot(elem["slot_name"], slot_model, elem["slot_value"])
        return True
    return False

### Testing

In [6]:
# def analyze_response(convo_context, slot_model, remaining_slots):
#     check_create_slot(convo_context, slot_model, remaining_slots)
#     check_fill_slot(convo_context, slot_model, remaining_slots)

In [7]:
# slot_model_in = ["departure_date", "budget", "return_date", "preferred_airline", "destination_city", "departure_city"]
# remaining_slots = {}
# convo_context_in = '''
#     System: Hello how may I assist you in booking a flight today?
#     User: I would like to book a flight to New York.
#     System: What is your departure city?
#     User: I am departing from Los Angeles.
#     System: What is your budget?
#     User: My budget is $500.
# '''

# analyze_response(convo_context_in, slot_model_in, remaining_slots)

In [8]:
# slot_model_in = ["preferred_airline"]
# remaining_slots = {}
# convo_context_in = '''
#     System: Hello how may I assist you in booking a flight today?
#     User: I would like to book a flight to New York.
#     System: What is your departure city?
#     User: I am departing from Los Angeles.
#     System: What is your budget?
#     User: My budget is $500.
#     System: When is your departure date?
#     User: I am departing on the 10th of November.
#     System: What is your return date?
#     User: I am returning on the 15th.
#     System: Is there anything else you would like to add?
#     User: Are you able to book a hotel for me?
# '''

# analyze_response(convo_context_in, slot_model_in, remaining_slots)

## response generation step

In [9]:
def generate_response(task, slot_name="", slot_value="", previous_response=""):
    if task == "regenerate":
        system_prompt = f'''
        '''
        user_prompt = f'''
        '''
        
        
    if task == "find_slot":
        system_prompt = f'''
            You are a system that will generate the next response based on the conversation context and a slot model.
            Generate a response that will prompt the user to provide the value for the slot name.
        '''
        user_prompt = f'''
            Conversation context: 
            {previous_response}
            
            Slot name: {slot_name}
        '''
        
    if task == "confirm":
        system_prompt = f'''
            
        '''
        user_prompt = f'''
        '''
    
    msgs_in = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
        
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=msgs_in,
    )
    
    return completion.choices[0].message.content
    
    

In [ ]:
def generate_summary(conversation, slot_model):
    system_prompt = f'''
        You are a system that will generate a response summarizing the information that has been collected.
        Generate a response that summarizes the information in the slot model.
        The slot model is an object of slot names and their corresponding values.
        You will also be given the conversation context that was used to collect the information.
        This summary message will be sent to the user to confirm the information that has been collected.
        Respond with only the summary message.
    '''
    user_prompt = f'''
        Conversation context: 
        {conversation}
        
        Slot model: 
        {slot_model}
        '''
    
    msgs_in = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
        
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=msgs_in,
    )
    
    return completion.choices[0].message.content

## Full pipeline

In [ ]:
def run_dialogue(start_state):
    slot_model = start_state
    remaining_slots = [key for key, value in slot_model.items() if value == "unfilled"]
    conversation = ""
    while remaining_slots:
        slot_name = remaining_slots[0]
        response = generate_response("find_slot", slot_name)
        conversation += f"System: {response}\n"
        print(response)
        user_response = input()
        conversation += f"User: {user_response}\n"
        created_slot = check_create_slot(conversation, slot_model, slot_model.keys())
        remaining_slots = [key for key, value in slot_model.items() if value == "unfilled"]
        filled_slot = check_fill_slot(conversation, slot_model, remaining_slots)
        # response = generate_response("confirm", slot_name, user_response)
        # print(response)
        remaining_slots = [key for key, value in slot_model.items() if value == "unfilled"]
        print(slot_model)
    summary_response = generate_summary(conversation, slot_model)
    print(summary_response)

In [ ]:
def run_dialogue_no_input():
    print("Enter a task request:")
    task_request = input()
    tools_in = [
        {
            "type": "function",
            "function": create_slot_func,
        }
    ]
    
    system_prompt = f'''
        Given a request from the user, determine what slots to create.
        A slot is a piece of information necessary to accomplish what the user wants.
        If the user asks a question or requests for information or an action then call create_slot_func and pass in a list of slot names designating the information the system would need answered by the user to accomplish what they request.
    '''
    
    user_prompt = f'''
        Here is the request:
        {task_request}
    '''
    
    msgs_in = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
        
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=msgs_in,
        tools=tools_in,
        temperature=0.3
    )
    
    tool_choice = completion.choices[0].message.tool_calls[0] if completion.choices[0].message.tool_calls else None
    
    slot_model = {}
    
    if tool_choice:
        arguments = json.loads(tool_choice.function.arguments)
        for slot in arguments["slots_to_add"]:
            create_slot(slot, slot_model)
    
    print(slot_model)
    run_dialogue(slot_model)

In [ ]:
start_slots = {"departure_date": "unfilled", "budget": "unfilled", "return_date": "unfilled", "preferred_airline": "unfilled", "destination_city": "unfilled", "departure_city": "unfilled"}
run_dialogue(start_slots)

            Could you please let me know the date you plan to depart?
{'departure_date': '11/20/24', 'budget': 'unfilled', 'return_date': 'unfilled', 'preferred_airline': 'unfilled', 'destination_city': 'unfilled', 'departure_city': 'unfilled'}

Sure! Could you please let me know what your budget is?
{'departure_date': '11/20/24', 'budget': '$500', 'return_date': 'unfilled', 'preferred_airline': 'unfilled', 'destination_city': 'unfilled', 'departure_city': 'unfilled', 'hotel_included': 'unfilled'}
When are you planning to return?
